In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import glob
import os

import src.FD_helper.visualization as visualize
import src.FD_helper.forward_euler_1D as FD_1D
import src.FD_helper.forward_euler_2D as FD_2D

figheight = 5

Transport phenomena is the study of the transferring of momentum, energy and mass. Oftentimes these processes are coupled as in boiling water, where bubbles of air trapped in the water nucleate due to the elevated temperature reducing the solubility of gases, followed by the density differences between the bubble and water causing the bubble to rise. These processes are further mediated through convection currents caused by temperature gradients in the boiling water. 

In this example, we will look at a far simple example of energy transport, characterized as the change in temperature of a heat source placed in an infinitely large cooler surrounding region. A visualization is shown in the figure below, where the red and blue regions indicate hot and cold areas respectively. Periodic boundary conditions will be assumed to overcome the limitation of finite sizes in compute.  

<img src="visualizations/system_setup.png" alt="drawing" width="500"/>

# Theory

For our problem, we will be basing our grid on rectilinear or cartesian coordinates. First, we will review the relevant transport equations. A more detailed explanation of each expression and derivation can be found in [Introductory Transport Phenomena](https://www.wiley.com/en-au/Introductory+Transport+Phenomena%2C+1st+Edition-p-9781118775523). We begin by writing a general energy transport equation through a control volume.

$$ change = accumulation + removal + generation + destruction $$

In our closed system, there is no generation, destruction and accumulation of energy, meaning that the change in energy over time can be written as

$ change = removal$

If we approximate that the energy lost from the addition of heat is lost through conduction, the removal of energy can be approximated using fourier's law, defined as 

$$ q = -k \nabla T $$

where $k (\frac{W}{mK})$ is the thermal conductivity of the matrix, $\nabla T (\frac{K}{m})$ is the temperature gradient between the heated area and its surroundings and $q (\frac{W}{m^2 K})$ is the heat flux through a surface. This is related to the energy change at each grid point over time as, 

$\frac{d E}{dt} = \nabla q$

Where $E (\frac{J}{m^3})$ is the energy density of a specific grid point and $t (s)$ is the time. This leads to the expression, 

$$ \frac{d E}{d t} = -k \nabla^2 T $$

The energy density of the heat source can be expressed from the heat capacity of the heat source, defined as $E = \rho c_p (T - T_{ref})$. If we make the further approximation that density $\rho (\frac{kg}{m^3})$ and specific heat capacity $\frac{J}{K kg}$ stay constant with temperature, we can rearrange the above expression to obtain the following PDE written as a function of temperature exclusively,

$$ \frac{dT}{dt} = -\frac{k}{\rho c_p} \nabla^2 T$$

Oftentimes, the factor $\frac{k}{\rho c_p}$ is condensed into the term $\alpha (\frac{m^2}{s})$ defining the thermal diffusivity of the system and will be used as an input parameter.

Now that we have laid out the basic theory being implemented, we shall turn to how we can convert math to something a computer can solve. There are derivatives of time and space that we will need to solve for. I will stick to the simplest ones here as the focus is on creating an illustration how the code looks different between implementations in different languages. You can learn more about different implementations from various textbooks such as [Applied Numerical Methods with MATLAB](https://www.mheducation.com/highered/product/Applied-Numerical-Methods-with-MATLAB-for-Engineers-and-Scientists-Chapra.html) for implementation details and [Numerical Methods for Scientific Computing](https://www.equalsharepress.com/media/NMFSC.pdf) for a rigorous mathematical understanding of the methods used.

## Finite difference

The simplest way to discretize a differential equation in space is using a finite difference technique. The core idea of this scheme is we convert the continous differential equations above into discrete sums and differences which we can solve. Below is a visualization as to what we do in a finite difference scheme. The grid is discretized into the points shown, and the red and blue regions are areas of high and low temperature.

<img src="visualizations/FDM_viz.png" alt="drawing" width="500"/>

$\Delta x$ and $\Delta y$ are the grid spacings in the $x$ and $y$ direction respectively. The derivatives are then expressed as a function of adjacent points. Commonly, this is done using a central difference scheme shown below for the first $f^1_x$ and second derivative $f^2_x$ for a function $f(x,y,t)$

\begin{align}
f^1_x &= \frac{f(x + \Delta x, y, t) - f(x - \Delta x, y, t)}{2\Delta x} \\
f^2_x &= \frac{f(x + \Delta x, y, t) - 2f(x,y,t) + f(x - \Delta x, y, t)}{\Delta x^2} 
\end{align}

The central difference discretization scheme produces second order error compared to forward or backward difference which produce first order error. 

## Forward Euler

Now that the derivative over space is explained, I explain how we perform timestepping or time integration. Time integration schemes  are split into explicit and implicit scheme categories. Explicit schemes include techniques such as Forward Euler and the Runge-Kutta time integrators while implicit schemes include the Backwards Euler and Crank-Nicholson schemes. Both categories solve for the dynamics of the system given an initial state of the system. For now, we will stick to the Forward Euler technique. Notationally, this looks like

\begin{align}
g(x, y, t_0) &= b \\
\frac{dg}{dt} &= f(x,y,t) \\
g(x, y , t + \Delta t) &= g(x, y, t) + \Delta t \frac{dg}{dt}
\end{align}

where $g(x, y, t)$ is a value of a function at time $t$ and location $(x,y)$, $\frac{dg}{dt}$ is the derivative of function $g$ with respect to time $t$ and is expressed as any function $f(x,y,t)$. The value of $g(x, y, t + \Delta t)$ is a sum of the value of $g(x, y, t)$ and the timestep over which the derivative $\frac{dg}{dt}$ is calculated.

The Forward Euler technique can be numerically unstable. Therefore care must be taken when selecting parameters for time and space integration and must fall within the CFL condition, defined as $C = \frac{u \Delta t}{\Delta x} < 1$. In this case as we do not have advection we will be using the CFL condition as defined from the thermal diffusivity, defined as $C = \frac{\alpha \Delta t}{\Delta x^2}$

## Python implementation

Using the established rules above, we move to generating a prototype in python. One could easily use [`scipy.integrate`](https://docs.scipy.org/doc/scipy/reference/integrate.html#solving-initial-value-problems-for-ode-systems) to access many of the time integration schemes used, but for the purpose of this exercise I will code everything up myself. `fun` used in [`scipy.integrate`](https://docs.scipy.org/doc/scipy/reference/integrate.html#solving-initial-value-problems-for-ode-systems) would be our spatial discretization using the central difference method. 

A 1D implementation is first shown to demonstrate the conceptual structure of the code. A heat source is placed in the middle of the system of length $L$, with periodic boundary conditions placed at each edge.

### 1D version

In [ ]:
## EDIT THIS ##
L = 256
dx = 1/L
hot_point = 64
halo = 1

dt = 0.0002
timesteps = 10000
dump_freq = 100

T_cold = 273
T_hot = 280
alpha = 0.002

bc_type = FD_1D.BoundaryType.PERIODIC

T_l = 275 # only does something if boundary type if set to constant_value
T_r = 275 # only does something if boundary type if set to constant_value
## EDIT THIS ##

C = alpha/(dx**2/dt)
if C > 1:
    raise RuntimeWarning("CFL condition not fulfilled")

In [ ]:
# creating data structures
rod_old = FD_1D.make_system(L, hot_point, T_hot, T_cold)

# inserting halo regions
rod_old = np.insert(rod_old, 0, 0)
rod_old = np.insert(rod_old, L+1, 0)

rod_new = rod_old.copy()
total_E = np.empty(timesteps + 1)

output_temperatures = np.empty((timesteps//dump_freq+1, L))
output_length = np.empty((timesteps//dump_freq+1, L))

for t in range(0, timesteps + 1):
    rod_old = FD_1D.apply_boundary(rod_old, bc_type, T_left=T_l, T_right=T_r, halo=halo)

    if t%dump_freq == 0:
        idx = t//dump_freq
        output_temperatures[idx] = rod_old[1:L+1]
        output_length[idx] = np.arange(0, L)
    
    rod_new = FD_1D.FD_timestep(rod_old, dx, dt, alpha, halo)

    total_E[t] = np.sum(rod_new[1:L])

    rod_old = rod_new.copy()

In [ ]:
ar = 1
fig, axs = plt.subplots(1, 2, figsize = (2*figheight*ar, figheight))

ax = axs[0]
ax.plot((total_E- total_E[0])/total_E[0]*100)

ax.set_ylabel("Total Energy")
ax.set_xlabel("t"+r"$\Delta t$")

ax = axs[1]
rod_original = FD_1D.make_system(L, hot_point, T_hot, T_cold)
ax.plot(rod_new, label = "Final")
ax.plot(rod_original, label = "Initial")
ax.set_ylim([T_cold, T_hot+1])


ax.set_ylabel("Temperature (K)")
ax.set_xlabel("Rod length"+r"$\Delta x$")
ax.legend()

fig.tight_layout()

The left plot shows the evolution of total system energy over time (normalized by the initial value), demonstrating near conservation initially followed by gradual energy loss at later times, indicative of discretization related errors. However these changes are near machine precision, indicating that we can disregard the effect on dynamics that these changes in energy may have. The right plot illustrates the temperature profile along the rod at the initial and final states. Initially, the temperature is localized in a hot square pulse at the center. Over time, thermal diffusion smooths the profile into a broader distribution.

Now that we have shown a succesful implementation in 1D, lets up the difficulty somewhat and move to 2D

### 2D version

In [ ]:
## CHANGE THESE ##
nx = 256
ny = 256
# keep square for now (nx == ny)
Lx = nx//4
Ly = ny//4
bc = FD_2D.BoundaryType.PERIODIC

dt = 0.000025
timesteps = 10000
dump_freq = 100

alpha = 0.01
T_h = 373 # temperature of hot region
T_c = 273 # temperature of cold region

T_l = 275 # temperature of left wall
T_r = 275 # temperature of right wall
T_t = 275 # temperature of top wall
T_b = 275 # temperature of bottom wall
## CHANGE THESE ##

halo = 1
dx = 1/nx
dy = 1/ny
C = alpha/(min(dx, dy)**2/dt)
if C > 0.1:
    raise RuntimeWarning("CFL condition not fulfilled")

In [ ]:
grid_old = FD_2D.make_system_square(nx+2*halo, ny+2*halo, Lx, Ly, T_h, T_c)
grid_new = np.empty((nx+2*halo, ny+2*halo))

temperature_fields_time = np.empty((timesteps//dump_freq+1, nx, ny))
times = np.empty(timesteps//dump_freq+1)
total_E = np.empty(timesteps//dump_freq+1)

for time in range(0, timesteps+1):
    grid_old = FD_2D.apply_boundary(grid_old, bc, T_left=T_l, T_right=T_r, T_top = T_t, T_bottom = T_b, halo=halo)

    if time%dump_freq == 0:
        temperature_fields_time[time//dump_freq] = grid_old[halo:nx+halo, halo:ny+halo]
        times[time//dump_freq] = time
        total_E[time//dump_freq] = np.sum(grid_old[halo:nx-halo,halo:ny-halo])
    
    grid_new = FD_2D.FD_timestep(grid_old, dx, dy, dt, alpha, halo)

    grid_old = grid_new.copy()

In [ ]:
ar = 1.25
fig, ar = plt.subplots(1, 1, figsize = (ar*figheight, figheight))
plt.plot(times, (total_E-total_E[0])/total_E[0]*100, "rx")

This plot shows the change in total energy over time for a 2D heat diffusion simulation using the implemented FDM scheme. The vertical axis represents the deviation of the system's total thermal energy from its initial value, and the horizontal axis represents time in discrete steps. The energy remains nearly constant although at later times, a small energy loss is observed. The magnitude of these differences being near machine precision indicate this behavior is attributable to accumulation of floating point roundoff and it not inherent to the technique. 

**Things to try**
1. Play around with different parameters(nx, ny, dt, alpha) and see when the simulation becomes numerically unstable

In [ ]:
ani = visualize.animate_colormap(temperature_fields_time, times = times, update_lims = False)
HTML(ani.to_jshtml())

We can see how the initial square of heat in the center of the domain begins to become circular, followed by a reduction in temperature from the outside in.

# Single core implementation in C++

Now that we have the base in python, lets implement the FD technique in C++. This section will use various `std` libraries in C++ but will omit MPI at the moment. We will be using pass by reference, although pass by value to more closely align with the python code can also be done, if slower.

You can play around with the settings of the implementation by editing the system through the sample inputs in the `main` function in `main.cpp`

```
// INPUT PARAMETERS
int nx = 1024; // size of field in x direction
int ny = 1024; // size of field in y direction

double dt = 0.000025; // timestep length 
double timesteps = 10000; // number of timesteps
int dump_freq = 10000; // Timesteps between data dump

double alpha = 0.002; // Thermal diffusivity of system
double T_hot = 373.0; // Temperature in K of the hot point
double T_cold = 273.0; // Temperature in K of the cold point
// INPUT PARAMETERS
```

Recompile and run once settings are changed to see them reflected in the simulation with these instructions.

```
make clean
make
./heat_solver
```

In [ ]:
path = "src/single_core"
data_paths = sorted(glob.glob(f"{path}/T*.txt"))
total_E = np.empty(len(data_paths))
times = np.empty(len(data_paths), dtype = int)
raw_data = []

for i, path in enumerate(data_paths):
    time = int(path.split("/")[-1].split("_")[-1].split(".")[0])
    data = visualize.read_data(path)

    times[i] = time
    raw_data.append(data)
    total_E[i] = np.sum(data)

raw_data = np.array(raw_data)

In [ ]:
ar = 1.25
fig, ax = plt.subplots(1, 1, figsize = (ar*figheight, figheight))

ax.plot(times, total_E/total_E[0], 'rx')
ax.set_ylabel(r"$\frac{E(t)}{E_0}$")
ax.set_xlabel(r"$t \Delta t$")

The implementation conserves energy to machine precision as expected.

In [ ]:
ani = visualize.animate_colormap(raw_data, times = times, update_lims = False)
HTML(ani.to_jshtml())

As in the 2D python implementation, the initial square of heat in the center of the domain begins to become circular, followed by a reduction in temperature from the outside in.

# Multicore (CPU) implementation in C++

2 main ways to perform compute on multiple cores are available. They are openMP and MPI based techniques. These differ on how memory is allocated between processors, which also impacts the scalability and performance of each based on system size and allocated resources. openMP uses shared memory to remove the need for message passing but is only supported by systems that share memory. MPI supports parallel computation for systems with distributed or shared memory as each process is executes independently from one another.

For personal computers, openMP offers a simple way to add parallelism without needing to resort to the extra preparation and planning needed to write and execute MPI based code at the expense of less flexibility if wanting to use HPC compute clusters such as the DoE's Frontier, Aurora or Perlmutter. 

Please read the source of my knowledge on this topic, the fantastic textbook [Parallel Programming in C with MPI and OpenMP](https://books.google.com/books/about/Parallel_Programming_in_C_with_MPI_and_O.html?id=YrgZAQAAIAAJ&source=kp_book_description). It may be slightly outdated given its 20+ years old, but it holds great information on the topic. I'm still learning new content as I read this textbook.

## OpenMP

This section is based on chapter 17 in the textbook "Parallel Programming in C with MPI and OpenMP" by Michael J. Quinn.

You can play around with the settings of the implementation by editing the system through the sample inputs in the `main` function in `main.cpp`

```
// INPUT PARAMETERS
const int threads = 2;
int nx = 1024; // size of field in x direction
int ny = 1024; // size of field in y direction

double dt = 0.000025; // timestep length 
double timesteps = 10000; // number of timesteps
int dump_freq = 10000; // Timesteps between data dump

double alpha = 0.002; // Thermal diffusivity of system
double T_hot = 373.0; // Temperature in K of the hot point
double T_cold = 273.0; // Temperature in K of the cold point
// INPUT PARAMETERS
```

Recompile and run once settings are changed to see them reflected in the simulation with these instructions.

```
make clean
make
./heat_solver
```

In [ ]:
path = "src/open_MP"
data_paths = sorted(glob.glob(f"{path}/T*.txt"))
total_E = np.empty(len(data_paths))
times = np.empty(len(data_paths), dtype = int)
raw_data = []

for i, path in enumerate(data_paths):
    time = int(path.split("/")[-1].split("_")[-1].split(".")[0])
    data = visualize.read_data(path)

    times[i] = time
    raw_data.append(data)
    total_E[i] = np.sum(data)

raw_data = np.array(raw_data)

In [ ]:
ar = 1.25
fig, ax = plt.subplots(1, 1, figsize = (ar*figheight, figheight))

ax.plot(times, total_E/total_E[0], 'rx')
ax.set_ylabel(r"$\frac{E(t)}{E_0}$")
ax.set_xlabel(r"$t \Delta t$")

Energy is conserved to machine precision.

In [ ]:
ani = visualize.animate_colormap(raw_data, times = times, update_lims = False)
HTML(ani.to_jshtml())

## OpenMPI

You can play around with the settings of the implementation by editing the system through the sample inputs in the `main` function in `main.cpp`

```
// INPUT PARAMETERS
int nx = 1024; // size of field in x direction
int ny = 1024; // size of field in y direction

double dt = 0.000025; // timestep length 
double timesteps = 10000; // number of timesteps
int dump_freq = 10000; // Timesteps between data dump

double alpha = 0.002; // Thermal diffusivity of system
double T_hot = 373.0; // Temperature in K of the hot point
double T_cold = 273.0; // Temperature in K of the cold point
// INPUT PARAMETERS
```

Recompile and run once settings are changed to see them reflected in the simulation with these instructions.

```
make clean
make
mpirun -n {n-cores} heat_solver
```

In [ ]:
path = "src/MPI"
data_paths = sorted(glob.glob(f"{path}/T*.txt"))
total_E = np.empty(len(data_paths))
times = np.empty(len(data_paths), dtype = int)
raw_data = []

for i, path in enumerate(data_paths):
    time = int(path.split("/")[-1].split("_")[-1].split(".")[0])
    data = visualize.read_data(path)

    times[i] = time
    raw_data.append(data)
    total_E[i] = np.sum(data)

raw_data = np.array(raw_data)

In [ ]:
ar = 1.25
fig, ax = plt.subplots(1, 1, figsize = (ar*figheight, figheight))

ax.plot(times, total_E/total_E[0], 'rx')
ax.set_ylabel(r"$\frac{E(t)}{E_0}$")
ax.set_xlabel(r"$t \Delta t$")

Energy is conserved to machine precision.

In [ ]:
ani = visualize.animate_colormap(raw_data, times = times, update_lims = False)
HTML(ani.to_jshtml())

## Scaling

The objective of any parallelized implementation is to reduce compute time. However, will this implementation succeed in this endeavour? To check whether it does, this section will analyze the strong and weak scaling. 

Strong scaling, compared against Amdahl's law, is measured through calculating the runtime as more resources are provided at the same system size. Broadly speaking, a system is said to scale strongly if there is a linear relationship between the observed speedup and the increase in the number of resources allocated. An expression commonly used to characterize this relationship is,

$$ S(n) = \frac{1}{(1 - P) + \frac{P}{n}} $$

Where $S$ is the speedup, $P$ is the proportion of the program that is parallelizable and $n$ is the number of cores or threads.

Weak scaling, compared against Gustafson's law, is measured through calculating the runtime as system size grows proportional with the resources allocated. A system can be said to scale weakly if there is a linear relationship between the increase in system size and the resources allocated to the program. An expression commonly used to characterize this relationship is

$$ S(n) = (1 - P) + Pn $$

To minimize the impact that writing output data has on the runtime, I only output data at the start and end of the simulation. I calculate the time taken for each part of the code using the following abbreviations

| Abbreviation | Measurement |
| :--: | :--: |
| IO | Time taken to write data |
| TS | Time taken for timestepping |
| BC | Time taken to perform boundary conditions |
| MP | Time taken to perform message passing |
| Tot | Total runtime |

My device uses an Intel i5 1345U with 8 E cores and 2 P cores. Each P core has 2 threads, limiting the maximum number of threads and cores I can use to 2 each. The performance of the code will vary depending on your system. The mean and standard deviation of runtimes are obtained from 3 runs in all scenarios.

### OpenMP

#### L = 256

##### Strong scaling

Strong scaling is compared to in this section using a $256^2$ system size run for $10^5$ timesteps. I will further breakdown the time taken for each part of the simulation based on the classification shown above. 

| Threads | TS (s) | BC (s) | IO (s) | Total (s) |
| :---:| :---: | :---: | :---: | :---: |
| 1 | 11.9505 $\pm$ 0.0200 | 0.1309 $\pm$ 0.0020 | 0.0315 $\pm$ 0.0025 | 12.1213 $\pm$ 0.0205 |
| 2 |  6.9363 $\pm$ 0.0620 | 0.1660 $\pm$ 0.0019 | 0.0291 $\pm$ 0.0005 |  7.1413 $\pm$ 0.0630 |

Speed up observed is approximately 41.1 \% which, while not the 50 \% expected represents a speedup. From the breakdown of timers, the IO and BC timings are mostly unaffected through the addition of more resources. This shows that the timestepping is the source of the lower than expected speedup.

**Things to try**
1. Investigate the effect that IO has on the parallel speedup observed.
2. If your system allows you to, use more cores to fit Amdahl's law and calculate the proportion of code that can be sped up through parallelization.

##### Weak scaling

Weak scaling going from 1 to 2 cpu cores are compared in this section. The simulation is run for $10^5$ timesteps. With 1 core, a system size of $256^2$ is used. With 2 cores, a system size of $256\cdot512$. I will further breakdown the time taken for each part of the simulation based on the classification shown above. 

| Threads | TS (s) | BC (s) | IO (s) | Total (s) |
| :---:| :---: | :---: | :---: | :---: |
| 1 | 11.9505 $\pm$ 0.0200 | 0.1309 $\pm$ 0.0020 | 0.0315 $\pm$ 0.0025 | 12.1213 $\pm$ 0.0205 |
| 2 | 14.4888 $\pm$ 0.0845 | 0.2271 $\pm$ 0.0030 | 0.0546 $\pm$ 0.0005 | 14.7811 $\pm$ 0.0820 |

Weak scaling shows that there is a slowdown of 21.9 \% when increasing system size and resource use. IO and BC are longer as the system size is doubled. The excess time arises from the increased duration of the timestepping step.

**Things to try**
1. Investigate the effect that IO has on the parallel speedup observed.
2. If your system allows you to, use more threads to fit Gustafson's law and calculate the proportion of code that is parallel. Is it the same as what is calculated when strong scaling? (Hint: It probably isn't)

#### L = 1024

##### Strong scaling

Strong scaling is compared to in this section using a $1024^2$ system size run for $10^4$ timesteps. I will further breakdown the time taken for each part of the simulation based on the classification shown above. 

| Threads | TS (s) | BC (s) | IO (s) | Total (s) |
| :---:| :---: | :---: | :---: | :---: |
| 1 | 20.1259 $\pm$ 0.2374 | 0.1582 $\pm$ 0.0008 | 0.3890 $\pm$ 0.0027 | 20.6819 $\pm$ 0.2413 |
| 2 | 11.5238 $\pm$ 0.3688 | 0.1025 $\pm$ 0.0073 | 0.3881 $\pm$ 0.0038 | 12.0228 $\pm$ 0.3783 |

Speed up observed is approximately 41.8 \% which, while not the 50 \% expected represents a speedup. From the breakdown of timers, the IO and BC timings are mostly unaffected through the addition of more resources. This shows that the timestepping is the source of the lower than expected speedup.

**Things to try**
1. Investigate the effect that IO has on the parallel speedup observed.
2. If your system allows you to, use more cores to fit Amdahl's law and calculate the proportion of code that can be sped up through parallelization.

##### Weak scaling

Weak scaling going from 1 to 2 threads are compared in this section. The simulation is run for $10^4$ timesteps. With 1 thread, a system size of $1024^2$ is used. With 2 threads, a system size of $1024\cdot2048$. I will further breakdown the time taken for each part of the simulation based on the classification shown above. 

| Threads | TS (s) | BC (s) | IO (s) | Total (s) |
| :---:| :---: | :---: | :---: | :---: |
| 1 | 20.1259 $\pm$ 0.2374 | 0.1582 $\pm$ 0.0008 | 0.3890 $\pm$ 0.0027 | 20.6819 $\pm$ 0.2413 |
| 2 | 22.6900 $\pm$ 0.0815 | 0.1463 $\pm$ 0.0054 | 0.7634 $\pm$ 0.0036 | 23.6172 $\pm$ 0.0909 |

Weak scaling shows that there is a slowdown of 14.2 \% when increasing system size and resource use. IO is longer as the system size is doubled. The excess time arises from the increased duration of the timestepping step.

**Things to try**
1. Investigate the effect that IO has on the parallel speedup observed.
2. If your system allows you to, use more threads to fit Gustafson's law and calculate the proportion of code that is parallel. Is it the same as what is calculated when strong scaling? (Hint: It probably isn't)

### MPI

#### L = 256

##### Strong scaling

Strong scaling is compared to in this section using a $256^2$ system size run for $10^5$ timesteps. I will further breakdown the time taken for each part of the simulation based on the classification shown above. 

| Cores | TS (s) | MP(s) | BC (s) | IO (s) | Total (s) |
| :---:| :---: | :---: | :---: | :---: | :---: |
| 1 | 11.8353 $\pm$ 0.0181 | 0.0583 $\pm$ 0.0013 | 0.0548 $\pm$ 0.0024 | 0.0299 $\pm$ 0.0018 | 11.9960 $\pm$ 0.0217 |
| 2 |  6.6084 $\pm$ 0.0457 | 0.4484 $\pm$ 0.0640 | 0.0221 $\pm$ 0.0005 | 0.0296 $\pm$ 0.0001 |  7.1242 $\pm$ 0.0739 |

Speed up observed is approximately 40.6 \% which, while not the 50 \% expected represents a speedup. The MP step is increased by almost 10x, while the TS step does not see the halving of compute time expected, resulting in the sub-optimal speedup characterized.

**Things to try**
1. Investigate the effect that IO has on the parallel speedup observed.
2. If your system allows you to, use more cores to fit Amdahl's law and calculate the proportion of code that can be sped up through parallelization.

##### Weak scaling

Weak scaling going from 1 to 2 cpu cores are compared in this section. The simulation is run for $10^5$ timesteps. With 1 core, a system size of $256^2$ is used. With 2 cores, a system size of $256\cdot512$. I will further breakdown the time taken for each part of the simulation based on the classification shown above. 

| Cores | TS (s) | MP(s) | BC (s) | IO (s) | Total (s) |
| :---:| :---: | :---: | :---: | :---: | :---: |
| 1 | 11.8353 $\pm$ 0.0181 | 0.0583 $\pm$ 0.0013 | 0.0548 $\pm$ 0.0024 | 0.0299 $\pm$ 0.0018 | 11.9960 $\pm$ 0.0217 |
| 2 | 13.3529 $\pm$ 0.2207 | 0.9945 $\pm$ 0.1183 | 0.0362 $\pm$ 0.0004 | 0.0612 $\pm$ 0.0039 | 14.4670 $\pm$ 0.1053 |

Weak scaling shows that there is a slowdown of 20.6 \% when increasing system size and resource use. IO is doubled, while BC is roughly halved. MP is almost 20x longer, indicating there is a significant loss of time from the halo exchange step.

**Things to try**
1. Investigate the effect that IO has on the parallel speedup observed.
2. If your system allows you to, use more threads to fit Gustafson's law and calculate the proportion of code that is parallel. Is it the same as what is calculated when strong scaling? (Hint: It probably isn't)

#### L = 1024

##### Strong scaling

Strong scaling is compared to in this section using a $1024^2$ system size run for $10^4$ timesteps. I will further breakdown the time taken for each part of the simulation based on the classification shown above. 

| Cores | TS (s) | MP(s) | BC (s) | IO (s) | Total (s) |
| :---:| :---: | :---: | :---: | :---: | :---: |
| 1 | 20.8662 $\pm$ 0.0982 | 0.0976 $\pm$ 0.0033 | 0.1683 $\pm$ 0.0084 | 0.3918 $\pm$ 0.0013 | 21.5452 $\pm$ 0.1055 |
| 2 | 12.0276 $\pm$ 0.2368 | 0.8574 $\pm$ 0.0736 | 0.0716 $\pm$ 0.0044 | 0.4064 $\pm$ 0.0054 | 13.3861 $\pm$ 0.1755 |

Speed up observed is approximately 38 \% which, while not the 50 \% expected represents a speedup. MP communication during the halo exchange step takes 10x longer than when running with 1 core, suggesting that as more cores are used this will represent a significant slowdown in performance.

**Things to try**
1. Investigate the effect that IO has on the parallel speedup observed.
2. If your system allows you to, use more cores to fit Amdahl's law and calculate the proportion of code that can be sped up through parallelization.

##### Weak scaling

Weak scaling going from 1 to 2 cpu cores are compared in this section. The simulation is run for $10^4$ timesteps. With 1 core, a system size of $1024^2$ is used. With 2 cores, a system size of $1024\cdot2048$. I will further breakdown the time taken for each part of the simulation based on the classification shown above. 

| Cores | TS (s) | MP(s) | BC (s) | IO (s) | Total (s) |
| :---:| :---: | :---: | :---: | :---: | :---: |
| 1 | 20.8662 $\pm$ 0.0982 | 0.0976 $\pm$ 0.0033 | 0.1683 $\pm$ 0.0084 | 0.3918 $\pm$ 0.0013 | 21.5452 $\pm$ 0.1055 |
| 2 | 21.6868 $\pm$ 0.0991 | 1.1872 $\pm$ 0.1845 | 0.0741 $\pm$ 0.0014 | 0.7730 $\pm$ 0.0190 | 23.7518 $\pm$ 0.2580 |

Weak scaling shows that there is a slowdown of 10.2 \% when increasing system size and resource use. This slowdown arises primarily from the increases duration of the MP step, representing the halo exchange taking disproportionately longer than at smaller system sizes. However compared to the L = 256 system which saw a 20x increase in MP time, here we only see a 10x increase in MP compute time. Additionally there is a smaller difference between TS when adding cores in the L = 1024 system compared to the L = 256.

**Things to try**
1. Investigate the effect that IO has on the parallel speedup observed.
2. If your system allows you to, use more cores to fit Gustafson's law and calculate the proportion of code that scales in parallel. Is it the same as what is calculated when strong scaling? (Hint: It probably isn't)

## Summary

The performance of the openMP and MPI implementations were compared using system sizes of L = 256 and L = 1024 using strong and weak scaling. Due to hardware limitations of my device, I could only test 1 and 2 threads/cores. 

The table below characterizes the strong scaling of each implementation when adding an additional thread or core.

| System size | Implementation  | Time change (\%) |
| :---: | :---: | :---: |
| 256 | openMP  | -41.1 \% |
| 256 | MPI | -40.6 \% |
| 1024 | openMP | -41.8 \%  |
| 1024 | MPI | -38 \%  |

The ideal speedup is 50 \%, showing that there is some performance loss when adding an additional core or thread at the same system size. However, the consistency in performance at small and large domains suggest that the computational cost of adding an additional thread or core are not dominant. At the core counts used, the code strong scales relatively well. 

The table below characterizes the weak scaling of each implementation when adding an additional thread or core.

| System size | Implementation  | Time change (\%) |
| :---: | :---: | :---: |
| 256 | openMP  | 21.9 \% |
| 256 | MPI | 20.6 \% |
| 1024 | openMP | 14.2 \%  |
| 1024 | MPI | 10.2 \%  |

The ideal time change should be 0 \%, showing that there is some performance loss when adding additional cores or threads. However the simulation slowdown is around 20 \% at L = 256 for both openMP and MPI implementations. At L = 1024, the slowdown reduces as the proportion of the system that requires inter-core or thread communication is smaller. 

# Improvements to MPI implementation

We shall use the L = 256 size system for this as we saw the greatest slowdown. Things we will try
1. Use non-blocking MPI communications
2. Use openMP and MPI at the same time
3. Use a more sophisticated domain decomposition strategy

We will time improvements at each step to characterize improvements in compute speed. We will also make memory access optimization throughout the code (e.g. removing multiple `center` definitions in timestepping)

## Non blocking MPI communication and threading

In this section, I will tackle easy to add enhancements to the code, namely the addition of non-blocking MPI communications using `MPI_Irecv, MPI_Isendv` and adding openMP to the MPI implementation. To control which implementation is compiled, use the flags `USE_SENDV_1D=0` for compiling with `MPI_Isendv` and `USE_OMP=1` for compiling with multithreading support. Instead of only using 3 repeats, here I run 100 samples and time each output with the same category of timers as before. As `IO` and `BC` are a relatively small part of the total time, I will only focus on `TS`, `MP` and `Total` times. 

In [ ]:
categories = ['TS', 'MP', "BC", "IO", "Total"]
colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

savedir = "src/MPI_mk2/"

blockings = ["blocking", "non_blocking"]
hybrids = ["non_hybrid", "hybrid"]
decomps = ["1D", "2D"]

datas = []
titles = []

for i, blocking in enumerate(blockings):
    for j, hybrid in enumerate(hybrids):
        titles.append(f"{blocking}-{hybrid}")
        text_file_path = f"{savedir}/profiling-{blocking}-{hybrid}-1D.txt"
        data = visualize.read_time_data(text_file_path)
        data = np.array(data)
        datas.append(data)

datas = np.array(datas)

In [ ]:
idx_of_focuss = [0, 1, -1]

ar = 2.1
sz = 3.5
fig, axs = plt.subplots(3, 1, figsize = (sz*ar, 3*sz))

for j, idx_of_focus in enumerate(idx_of_focuss):
    for i in range(len(datas)):
        ax = axs[j]
        curr_data = datas[i, :, idx_of_focus]
        ax.violinplot(curr_data, positions = [i], showmeans=True, showextrema = False)
        # print(round(curr_data.mean(), 4), round(curr_data.std(), 4))
    ax.set_title(f"{categories[idx_of_focus]}")
    ax.set_xticks(np.arange(0, len(datas)))
    ax.set_xticklabels(titles)
    ax.set_ylabel('Execution Time (s)')

fig.tight_layout()

| Comms | Threaded | TS (s) | MP(s) | Total (s) |
| :---:|         :---: |:---:                | :---:               | :---:               |
| Blocking |     False | 7.4193 $\pm$ 0.6347 | 0.6395 $\pm$ 0.5403 | 8.1330 $\pm$ 0.8274 |
| Blocking |     True  | 7.5082 $\pm$ 0.3942 | 0.6040 $\pm$ 0.2837 | 8.1873 $\pm$ 0.4717 |
| Non-blocking | False | 7.4047 $\pm$ 0.2858 | 0.5442 $\pm$ 0.2143 | 8.0224 $\pm$ 0.3547 |
| Non-blocking | True  | 7.0324 $\pm$ 0.3956 | 0.5007 $\pm$ 0.1913 | 7.6037 $\pm$ 0.4511 |

The results, as visualized in the violin plots and summarized in the accompanying table, provide a comprehensive view of how communication strategy and parallelism affect the heat solver’s performance. Each violin plot reflects the distribution of execution times across 100 runs for the time-stepping (TS), halo exchange (MP), and total runtime.

In the TS plot, despite this phase containing no explicit communication, the distributions reveal a consistent that non-blocking implementations achieve both lower average times and smaller variances, indicating better computational efficiency and more consistent performance. The blocking variants, in contrast, show wider distributions suggesting higher variability, likely due to delayed transitions from prior communication phases and less efficient CPU scheduling. The addition of multi-threading using openMP improves performance in the non-blocking communication strategy but paradoxically seems to reduce performance when using a blocking strategy. This shows that asynchronous communication which facilitates simultaneous compute and communication between cores reduces communication costs.

Similar trends can be seen in the halo exchange (MP) and total rutime (total) plots. The non-blocking hybrid configuration shows the shortest and least varying compute time. In contrast, the blocking non-hybrid approach is both the slowest and most unstable, suffering from high execution time variability and inefficiencies in both communication and computation.

Together, the violin plots visually reinforce that simultaneous communication and compute allows for more robust and performant parallelized code, reducing both mean runtime and performance variance. This highlights the importance of optimizing not just algorithms, but also parallel execution strategies, especially in high-performance computing environments.

## Redoing the domain decomposition

In [ ]:
path = "src/MPI_mk2"
data_paths = sorted(glob.glob(f"{path}/T*.txt"))
total_E = np.empty(len(data_paths))
times = np.empty(len(data_paths), dtype = int)
raw_data = []

for i, path in enumerate(data_paths):
    time = int(path.split("/")[-1].split("_")[-1].split(".")[0])
    data = visualize.read_data(path)

    times[i] = time
    raw_data.append(data)
    total_E[i] = np.sum(data)

raw_data = np.array(raw_data)

In [ ]:
ar = 1.25
fig, ax = plt.subplots(1, 1, figsize = (ar*figheight, figheight))

ax.plot(times, total_E/total_E[0], 'rx')
ax.set_ylabel(r"$\frac{E(t)}{E_0}$")
ax.set_xlabel(r"$t \Delta t$")

In [ ]:
categories = ['TS', 'MP', "BC", "IO", "Total"]
colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

savedir = "src/MPI_mk2/"

blockings = ["blocking", "non_blocking"]
hybrids = ["non_hybrid", "hybrid"]
decomps = ["1D", "2D"]

datas = []
titles = []

for i, blocking in enumerate(blockings):
    for j, hybrid in enumerate(hybrids):
        for k, decomp in enumerate(decomps):
            title = f"{blocking}\n{hybrid}\n{decomp}"
            text_file_path = f"{savedir}/profiling-{blocking}-{hybrid}-{decomp}.txt"
            if os.path.exists(text_file_path):
                titles.append(title)
                data = visualize.read_time_data(text_file_path)
                data = np.array(data)
                datas.append(data)

datas = np.array(datas)

In [ ]:
idx_of_focuss = [0, 1, -1]

ar = 2.1
sz = 3.5
fig, axs = plt.subplots(len(idx_of_focuss), 1, figsize = (sz*ar, len(idx_of_focuss)*sz))

for j, idx_of_focus in enumerate(idx_of_focuss):
    for i in range(len(datas)):
        ax = axs[j]
        curr_data = datas[i, :, idx_of_focus]
        ax.violinplot(curr_data, positions = [i], showmeans=True, showextrema = False)
        print(round(curr_data.mean(), 4), round(curr_data.std(), 4))
    ax.set_title(f"{categories[idx_of_focus]}")
    ax.set_xticks(np.arange(0, len(datas)))
    ax.set_xticklabels(titles)
    ax.set_ylabel('Execution Time (s)')

fig.tight_layout()

In [ ]:
curr_data